# Laboratorium 5
## ADAM GAWRYŚ, DAWID KLEINA

## Zadanie 1. (0 pkt.)
Zbiór danych: https://archive.ics.uci.edu/ml/datasets/SmartphoneBased+Recognition+of+Human+Activities+and+Postural+Transitions

• Treningowy: X_train.txt / y_train.txt

• Testowy: X_test.txt / y_test.txt

In [1]:
import numpy as np
import pandas as pd 
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import collections
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, log_loss
from sklearn.model_selection import train_test_split, KFold, cross_val_score

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
data_train = pd.read_csv("X_train.txt", sep=" ", header = None)
result_train = pd.read_csv("y_train.txt", sep=" ", header = None)
data_test = pd.read_csv("X_test.txt", sep=" ", header = None)
result_test = pd.read_csv("y_test.txt", sep=" ", header = None)

In [3]:
data_train = pd.DataFrame(data=data_train)
result_train = pd.DataFrame(data=result_train)
data_test = pd.DataFrame(data=data_test)
result_test = pd.DataFrame(data=result_test)

data_train.reset_index(drop=True)
result_train.reset_index(drop=True)
data_test.reset_index(drop=True)
result_test.reset_index(drop=True)

,0
0,5
1,5
2,5
3,5
4,5
...,...
3157,2
3158,2
3159,2
3160,2


In [4]:
X_train = data_train
y_train = result_train
X_test = data_test
y_test = result_test

In [5]:
y_train = np.resize(y_train, (7767,))
y_train.shape

(7767,)

In [6]:
y_test = np.resize(y_test, (3162,))
y_test.shape

(3162,)

### Zadanie 2. (3 pkt.)
Wykorzystując zbiór danych z poprzednich laboratoriów wykonaj uczenie zespołowe dla w
następującej konfiguracji:
1. Liczba klasyfikatorów: 3
2. Algorytmy klasyfikacji: dowolne
3. Trenowanie: każdy klasyfikator na całym zbiorze treningowym

W ramach uczenia zespołowego wypisz wypadkową ocenę stosując zasadę większości
głosów. 

Jako rezultat przedstaw wynik kroswalidacji dla 5 podprób oraz wynik testowania na zbiorze
testowym. 

Jako kryterium oceny klasyfikacji wypisz następujące parametry:

• ACC

• Recall

• F1

• AUC

Rezultaty zapisz do pliku o nazwie ensemble_learning z rozszerzeniem xls lub xlsx.

In [7]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
DTree_clf = DecisionTreeClassifier()
svm_1 = svm.SVC(decision_function_shape='ovr')


knn.fit(X_train, y_train)
DTree_clf.fit(X_train, y_train)
svm_1.fit(X_train, np.ravel(y_train,order='C'))

knn_pred= knn.predict(X_test)
DTree_pred = DTree_clf.predict(X_test)
SVC_pred = svm_1.predict(X_test)

averaged_preds = (DTree_pred + SVC_pred + knn_pred)//3
acc = accuracy_score(y_test, averaged_preds)
print(acc)

0.8526249209361164


In [8]:
voting_clf = VotingClassifier(estimators=[('SVC', svm_1), ('DTree', DTree_clf), ('knn', knn)], voting='hard')
voting_clf.fit(X_train, y_train)
preds_2_1 = voting_clf.predict(X_test)
report = classification_report(y_test, preds_2_1, output_dict=True)

print(confusion_matrix(y_test, preds_2_1))
print(classification_report(y_test, preds_2_1))

[[489   4   3   0   0   0   0   0   0   0   0   0]
 [ 31 436   4   0   0   0   0   0   0   0   0   0]
 [ 23  34 363   0   0   0   0   0   0   0   0   0]
 [  0   4   0 442  62   0   0   0   0   0   0   0]
 [  0   1   0  33 522   0   0   0   0   0   0   0]
 [  0   1   0   0   0 544   0   0   0   0   0   0]
 [  1   1   0   1   0   0  19   0   1   0   0   0]
 [  0   0   0   0   0   0   0  10   0   0   0   0]
 [  0   0   0   0   0   0   0   0  27   0   5   0]
 [  0   0   0   0   0   0   0   0   1  19   0   5]
 [  3   3   0   1   1   1   0   0  12   0  28   0]
 [  1   0   0   0   0   0   2   0   0   9   1  14]]
              precision    recall  f1-score   support

           1       0.89      0.99      0.94       496
           2       0.90      0.93      0.91       471
           3       0.98      0.86      0.92       420
           4       0.93      0.87      0.90       508
           5       0.89      0.94      0.91       556
           6       1.00      1.00      1.00       545
        

In [45]:
df = pd.DataFrame(report).transpose()
df.rename(columns = {'precision' : 'precision_ensemble', 'recall' : 'recall_ensemble', 'f1-score' : 'f1-score_ensemble', 'support' : 'support_ensemble'}, inplace = True)

In [46]:
df.to_excel("ensemble_learning.xlsx") 

## Zadanie 3. (3 pkt.)
Wykorzystując zbiór danych z poprzednich laboratoriów wykonaj uczenie zespołowe stosując agregację klasyfikatorów dla w następującej konfiguracji:
4. Liczba klasyfikatorów: 3
5. Algorytmy klasyfikacji: dowolne
6. Trenowanie: na losowo wygenerowanych podzbiorach

W ramach agregacji klasyfikatorów wypisz wypadkową ocenę stosując zasadę większości głosów.

Jako rezultat przedstaw wynik kroswalidacji dla 5 podprób oraz wynik testowania na zbiorze testowym.

Jako kryterium oceny klasyfikacji wypisz następujące parametry:

• ACC

• Recall

• F1

• AUC

Rezultaty zapisz do pliku o nazwie aggregating z rozszerzeniem xls lub xlsx.


In [31]:
from sklearn.ensemble import BaggingClassifier

In [32]:
knn_bagging_model = BaggingClassifier(base_estimator=knn)
dtree_bagging_model = BaggingClassifier(base_estimator=DTree_clf, n_estimators=5)
random_forest = RandomForestClassifier(n_estimators=5)


def bagging_ensemble(model):
    k_folds = KFold(n_splits=5)
    results = cross_val_score(model, X_train, y_train, cv=k_folds)
    print(results.mean())

bagging_ensemble(knn_bagging_model)
bagging_ensemble(dtree_bagging_model)
bagging_ensemble(random_forest)



0.8842546621683776
0.866100402674775
0.866744897781601


In [33]:
voting_clf2 = VotingClassifier(estimators=[('Rf', random_forest), ('DTree', dtree_bagging_model), ('knn', knn_bagging_model)], voting='hard')
voting_clf2.fit(X_train, y_train)
preds_3_2 = voting_clf2.predict(X_test)
report = classification_report(y_test, preds_3_2, output_dict=True)

print(confusion_matrix(y_test, preds_3_2))
print(classification_report(y_test, preds_3_2))

[[487   2   7   0   0   0   0   0   0   0   0   0]
 [ 56 410   5   0   0   0   0   0   0   0   0   0]
 [ 29  43 348   0   0   0   0   0   0   0   0   0]
 [  0   2   0 423  82   0   1   0   0   0   0   0]
 [  0   1   0  39 516   0   0   0   0   0   0   0]
 [  0   1   0   0   0 544   0   0   0   0   0   0]
 [  1   1   0   3   0   0  17   0   1   0   0   0]
 [  0   0   0   2   0   0   2   6   0   0   0   0]
 [  0   2   0   1   0   0   0   0  23   0   6   0]
 [  0   1   0   0   0   0   0   0   1  18   1   4]
 [  3   5   0   1   1   0   1   0   9   0  29   0]
 [  1   1   0   0   0   0   0   1   0   9   1  14]]
              precision    recall  f1-score   support

           1       0.84      0.98      0.91       496
           2       0.87      0.87      0.87       471
           3       0.97      0.83      0.89       420
           4       0.90      0.83      0.87       508
           5       0.86      0.93      0.89       556
           6       1.00      1.00      1.00       545
        

In [47]:
reportbagg = classification_report(y_test, preds_3_2, output_dict=True)
df3_1 = pd.DataFrame(reportbagg).transpose()
df3_1.rename(columns = {'precision' : 'precision_bagg', 'recall' : 'recall_bagg', 'f1-score' : 'f1-score_bagg', 'support' : 'support_bagg'}, inplace = True)

In [48]:
df3_1.to_excel("aggregating.xlsx") 

## Zadanie 3. (3 pkt.)

Wykonaj tożsame zadanie klasyfikacji jak w poprzednich ćwiczeniach dla następujących metod uczenia zespołowego przyjmując domyślne parametry:

• ADABoost

• XGBoost

Rezultaty zapisz do pliku o nazwie boosting z rozszerzeniem xls lub xlsx.


In [11]:
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier()
abc.fit(X_train, y_train)
y_pred_abc = abc.predict(X_test)

In [12]:
print(confusion_matrix(y_test, y_pred_abc ))
print(classification_report(y_test, y_pred_abc ))

[[475   0   0   0   0   0   0   0   0   0  21   0]
 [464   0   0   0   0   0   0   0   0   0   7   0]
 [420   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0 499   0   0   0   0   0   9   0]
 [  0   0   0   0 553   0   0   0   0   0   3   0]
 [  0   0   0   0   0 545   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0  23   0]
 [  0   0   0   0   0   0   0   0   0   0  10   0]
 [  0   0   0   0   0   1   0   0   0   0  31   0]
 [  0   0   0   0   0   1   0   0   0  23   1   0]
 [  0   0   0   0   1   1   0   0   0   0  47   0]
 [  0   0   0   0   0   2   0   0   0  22   3   0]]
              precision    recall  f1-score   support

           1       0.35      0.96      0.51       496
           2       0.00      0.00      0.00       471
           3       0.00      0.00      0.00       420
           4       0.00      0.00      0.00       508
           5       0.53      0.99      0.69       556
           6       0.99      1.00      1.00       545
        

C:\Users\malgo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [49]:
reportada = classification_report(y_test, y_pred_abc, output_dict=True)
df3_21 = pd.DataFrame(reportada).transpose()
df3_21.rename(columns = {'precision' : 'precision_ada', 'recall' : 'recall_ada', 'f1-score' : 'f1-score_ada', 'support' : 'support_ada'}, inplace = True)

C:\Users\malgo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
from xgboost import XGBClassifier

xgbc = XGBClassifier()
xgbc.fit(X_train, y_train)
y_pred_xgbc = xgbc.predict(X_test)

C:\Users\malgo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:27:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [17]:
print(confusion_matrix(y_test, y_pred_xgbc ))
print(classification_report(y_test, y_pred_xgbc ))

[[482  10   4   0   0   0   0   0   0   0   0   0]
 [ 34 430   6   0   0   0   0   0   0   0   1   0]
 [  9  38 373   0   0   0   0   0   0   0   0   0]
 [  0   0   0 442  64   0   0   1   1   0   0   0]
 [  0   0   0  34 521   0   1   0   0   0   0   0]
 [  0   0   0   0   0 545   0   0   0   0   0   0]
 [  0   3   0   1   0   0  18   1   0   0   0   0]
 [  0   0   0   0   0   0   0  10   0   0   0   0]
 [  0   0   0   0   0   0   0   0  24   0   8   0]
 [  0   0   0   0   0   0   0   1   0  15   0   9]
 [  2   0   0   2   0   0   0   1   7   0  36   1]
 [  0   0   0   0   0   0   1   1   0   8   1  16]]
              precision    recall  f1-score   support

           1       0.91      0.97      0.94       496
           2       0.89      0.91      0.90       471
           3       0.97      0.89      0.93       420
           4       0.92      0.87      0.90       508
           5       0.89      0.94      0.91       556
           6       1.00      1.00      1.00       545
        

In [50]:
reportxgb = classification_report(y_test, y_pred_xgbc, output_dict=True)
df3_22 = pd.DataFrame(reportxgb).transpose()
df3_22.rename(columns = {'precision' : 'precision_xgb', 'recall' : 'recall_xgb', 'f1-score' : 'f1-score_xgb', 'support' : 'support_xgb'}, inplace = True)

In [51]:
result = pd.concat([df3_21, df3_22], axis=1, join='inner')
result

,precision_ada,recall_ada,f1-score_ada,support_ada,precision_xgb,recall_xgb,f1-score_xgb,support_xgb
1,0.349522,0.957661,0.512129,496.000000,0.914611,0.971774,0.942326,496.000000
2,0.000000,0.000000,0.000000,471.000000,0.893971,0.912951,0.903361,471.000000
3,0.000000,0.000000,0.000000,420.000000,0.973890,0.888095,0.929016,420.000000
4,0.000000,0.000000,0.000000,508.000000,0.922756,0.870079,0.895643,508.000000
5,0.525166,0.994604,0.687383,556.000000,0.890598,0.937050,0.913234,556.000000
6,0.990909,1.000000,0.995434,545.000000,1.000000,1.000000,1.000000,545.000000
7,0.000000,0.000000,0.000000,23.000000,0.900000,0.782609,0.837209,23.000000
8,0.000000,0.000000,0.000000,10.000000,0.666667,1.000000,0.800000,10.000000
9,0.000000,0.000000,0.000000,32.000000,0.750000,0.750000,0.750000,32.000000
10,0.511111,0.920000,0.657143,25.000000,0.652174,0.600000,0.625000,25.000000


In [26]:
result.to_excel("boosting.xlsx") 

## Zadanie 4. (1 pkt.)
Porównaj wyniki klasyfikacji wszystkich metod uczenia zespołowego w zbiorczej tabeli na podstawie następujących kryteriów:

• ACC

• Recall

• F1

• AUC

Rezultaty zapisz do pliku o nazwie comparison z rozszerzeniem xls lub xlsx.

In [56]:
XX = pd.concat([df, df3_1, result], axis=1, join='inner')
XX

,precision_ensemble,recall_ensemble,f1-score_ensemble,support_ensemble,precision_bagg,recall_bagg,f1-score_bagg,support_bagg,precision_ada,recall_ada,f1-score_ada,support_ada,precision_xgb,recall_xgb,f1-score_xgb,support_xgb
1,0.844021,0.981855,0.907735,496.000000,0.844021,0.981855,0.907735,496.000000,0.349522,0.957661,0.512129,496.000000,0.914611,0.971774,0.942326,496.000000
2,0.874200,0.870488,0.872340,471.000000,0.874200,0.870488,0.872340,471.000000,0.000000,0.000000,0.000000,471.000000,0.893971,0.912951,0.903361,471.000000
3,0.966667,0.828571,0.892308,420.000000,0.966667,0.828571,0.892308,420.000000,0.000000,0.000000,0.000000,420.000000,0.973890,0.888095,0.929016,420.000000
4,0.901919,0.832677,0.865916,508.000000,0.901919,0.832677,0.865916,508.000000,0.000000,0.000000,0.000000,508.000000,0.922756,0.870079,0.895643,508.000000
5,0.861436,0.928058,0.893506,556.000000,0.861436,0.928058,0.893506,556.000000,0.525166,0.994604,0.687383,556.000000,0.890598,0.937050,0.913234,556.000000
6,1.000000,0.998165,0.999082,545.000000,1.000000,0.998165,0.999082,545.000000,0.990909,1.000000,0.995434,545.000000,1.000000,1.000000,1.000000,545.000000
7,0.809524,0.739130,0.772727,23.000000,0.809524,0.739130,0.772727,23.000000,0.000000,0.000000,0.000000,23.000000,0.900000,0.782609,0.837209,23.000000
8,0.857143,0.600000,0.705882,10.000000,0.857143,0.600000,0.705882,10.000000,0.000000,0.000000,0.000000,10.000000,0.666667,1.000000,0.800000,10.000000
9,0.676471,0.718750,0.696970,32.000000,0.676471,0.718750,0.696970,32.000000,0.000000,0.000000,0.000000,32.000000,0.750000,0.750000,0.750000,32.000000
10,0.666667,0.720000,0.692308,25.000000,0.666667,0.720000,0.692308,25.000000,0.511111,0.920000,0.657143,25.000000,0.652174,0.600000,0.625000,25.000000


In [57]:
XX.to_excel("comparison.xlsx") 

### Wnioski: 
Niniejsze laboratorium było bardzo ważne pod względem analizy statystycznej w porównaniu wyników klasyfikacji wszsystkich przeprowadzonych metod uczenia zespołowego. Jako że te zadania były bardzo cenne jeśli chodzi o zapoznanie się wynikami metryk dla poszczególych klasyfikacji pozwoliło to nam na lepsze zapoznanie się z tą dziedziną oceny. Same wyniki są mocno zróżnicowane i wyniki przy różnych metrykach są znacznie inne dla odpowiedniej liczby podprób.
Nową rzeczą było również samo zapisywanie w formacie xls, gdyż wcześniej tego nie robiliśmy. 